# BigQuery Release Note Notifier

This notebook automates the process of identifying new BigQuery release notes, generating information to evaluate the new feature and learn more, and then posting the suggestions to a Google Chat space.

This version is specifically designed for **BigQuery Studio** and uses Google Cloud's native authentication (ADC) and Secret Manager.

<table align="left">
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/aliciawilliams/snippets/blob/master/notebooks/bq_releases_chat_notification.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery Studio logo"><br> Open in BigQuery Studio
    </a>
  </td>
</table>

## ⚙️ 1. Setup and Installation

First, install the necessary Python libraries from PyPI.

In [21]:
!pip install google-genai google-cloud-secret-manager google-cloud-bigquery python-dateutil beautifulsoup4 lxml --quiet

## 📚 2. Import Libraries

In [ ]:
import os
import json
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta, timezone
from dateutil import parser
from bs4 import BeautifulSoup
import hashlib
import re

# Import the necessary Google Cloud libraries
from google import genai
from google.genai.types import HttpOptions
from google.cloud import secretmanager
from google.cloud import bigquery

## ☁️ 3. Configure Google Cloud Environment

Initialize the Google Gen AI SDK. In Colab Enterprise, it will automatically use the project and credentials from the environment. Then, set the variables to retrieve your Google Chat webhook URL securely from Secret Manager.

In [ ]:
# --- Configuration ---
GCP_PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]
CHAT_WEBHOOK_SECRET_ID = "chat-webhook-url"
LOOKBACK_DAYS = 7
BIGQUERY_DATASET_ID = "release_note_tracker"
BIGQUERY_TABLE_ID = "processed_features"
LOCATION = "global"

In [ ]:
client = genai.Client(vertexai=True, project=GCP_PROJECT_ID, location=LOCATION)
model_id = "gemini-2.5-flash"

In [ ]:
# Comment out or delete this cell once Dataset is created
!bq mk --location=US --dataset $GCP_PROJECT_ID:$BIGQUERY_DATASET_ID

In [ ]:
# Construct the DDL query using the variables
ddl_query = f"""
CREATE TABLE IF NOT EXISTS `{GCP_PROJECT_ID}.{BIGQUERY_DATASET_ID}.{BIGQUERY_TABLE_ID}` (
    feature_id STRING NOT NULL OPTIONS(description="A unique hash of the feature content to prevent duplicates"),
    published_date DATE OPTIONS(description="The publication date of the release note"),
    release_title STRING OPTIONS(description="The title of the parent release note"),
    feature_description STRING OPTIONS(description="The HTML content of the specific feature"),
    gemini_suggestion STRING OPTIONS(description="The suggestion generated by Gemini"),
    processing_timestamp TIMESTAMP NOT NULL OPTIONS(description="When the feature was processed by this notebook")
)
"""

# Execute the query using the bq command-line tool
!bq query --nouse_legacy_sql '{ddl_query}'

In [ ]:
def get_secret(secret_id, project_id, version_id="latest"):
    """
    Retrieves a secret from Google Cloud Secret Manager.
    """
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    try:
        response = client.access_secret_version(request={"name": name})
        return response.payload.data.decode("UTF-8")
    except Exception as e:
        print(f"❌ Error retrieving secret: {e}")
        print("Please ensure the secret exists and the service account has the 'Secret Manager Secret Accessor' role.")
        return None

# Fetch the webhook URL at the start of the script.
CHAT_WEBHOOK_URL = get_secret(CHAT_WEBHOOK_SECRET_ID, GCP_PROJECT_ID)

## 📰 4. Define Helper Functions

In [ ]:
def get_recent_release_notes(days=1):
    """
    Fetches BigQuery release notes, extracts individual features marked with
    <h3>Feature</h3>, and returns each feature as a separate item.
    """
    feed_url = "https://cloud.google.com/feeds/bigquery-release-notes.xml"
    recent_features = [] # This list will hold individual features
    time_threshold = datetime.now(timezone.utc) - timedelta(days=days)

    print(f"Fetching notes published after {time_threshold.strftime('%Y-%m-%d %H:%M:%S %Z')}...")

    try:
        response = requests.get(feed_url)
        response.raise_for_status()
        root = ET.fromstring(response.content)
        namespaces = {'atom': 'http://www.w3.org/2005/Atom'}

        for entry in root.findall('atom:entry', namespaces):
            date_element = entry.find('atom:published', namespaces)
            if date_element is None:
                date_element = entry.find('atom:updated', namespaces)

            if date_element is not None:
                published_str = date_element.text
                published_date = parser.parse(published_str)

                if published_date >= time_threshold:
                    base_title = entry.find('atom:title', namespaces).text
                    base_link = entry.find('atom:link', namespaces).get('href')
                    base_published_date = published_date.strftime("%Y-%m-%d")

                    content_element = entry.find('atom:summary', namespaces) or entry.find('atom:content', namespaces)
                    html_content = content_element.text if content_element is not None else ""

                    if not html_content:
                        continue

                    soup = BeautifulSoup(html_content, 'lxml')
                    h3_tags = soup.find_all('h3')

                    for h3 in h3_tags:
                        # Process only if the h3 tag is for a "Feature"
                        if h3.get_text(strip=True).lower() == 'feature':
                            feature_html_parts = []
                            # Collect all sibling tags after this h3 until the next h3
                            for sibling in h3.find_next_siblings():
                                if sibling.name == 'h3':
                                    break # Stop when we hit the next section
                                feature_html_parts.append(str(sibling))

                            feature_description = "".join(feature_html_parts).strip()

                            if feature_description:
                                # Create a separate record for this specific feature
                                recent_features.append({
                                    'title': base_title,
                                    'link': base_link,
                                    'published': base_published_date,
                                    'summary': feature_description
                                })

    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching the URL: {e}")
    except ET.ParseError as e:
        print(f"❌ Error parsing the XML: {e}")

    return recent_features

In [ ]:
def generate_use_cases(release_note):
    """
    Generates onboarding suggestions for a new feature using Gemini.
    """
    prompt = f"""
    You are an AI assistant specialized in helping data analysts quickly understand and apply new technologies.
    Your goal is to provide practical, actionable advice about the new BigQuery feature described below.

    Instructions:
    1.  **Where to Learn More:** Briefly suggest 1-2 key resources where an analyst can find detailed information or examples (e.g., official documentation, a specific tutorial, or a relevant blog post). Use the provided link as your primary source.
    2.  **How to Investigate its Usefulness:** Outline three distinct and practical ways a data analyst can quickly determine if this new feature could be valuable to them.
        - Frame these as concise "Consider this if..." or "Try this if..." scenarios that connect directly to common data analyst tasks and pain points.
        - The goal is to prompt self-reflection on their current workflow, not to propose a full project.
    -   Be direct and concise. Avoid conversational introductions or summaries.

    *** NEW SECTION START ***
    Formatting Instructions:
    -   **You MUST format your entire response using simple HTML tags.**
    -   Use `<b>` for bold headers (e.g., `<b>Where to Learn More:</b>`).
    -   Use `<br>` for all line breaks.
    -   Use `<a>` tags for any links.
    -   Use "• " (a bullet and a space) for bullet points, followed by a `<br>`.
    -   Do NOT use Markdown (like `**`, `*`, or `- ` lists).
    *** NEW SECTION END ***

    Release Note Title: "{release_note['title']}"
    Link to full note: {release_note['link']}

    Feature to Analyze:
    {release_note['summary']}

    Generate the HTML-formatted actionable advice now.
    """
    try:
        response = client.models.generate_content(model=model_id,contents=prompt)
        return response.text
    except Exception as e:
        print(f"❌ Error generating content with Gemini: {e}")
        print("Please ensure the Vertex AI API is enabled and the service account has the 'Vertex AI User' role.")
        return None

In [ ]:
def get_processed_ids_from_bq(project_id, dataset_id, table_id):
    """Queries BigQuery to get the set of all feature_id's already processed."""
    bq_client = bigquery.Client(project=project_id)
    table_ref = f"{project_id}.{dataset_id}.{table_id}"
    query = f"SELECT feature_id FROM `{table_ref}`"

    try:
        query_job = bq_client.query(query)
        processed_ids = {row.feature_id for row in query_job}
        print(f"Loaded {len(processed_ids)} processed feature IDs from BigQuery.")
        return processed_ids
    except Exception as e:
        print(f"⚠️ Could not query BigQuery table. Assuming no prior history. Error: {e}")
        return set()

In [ ]:
def save_result_to_bq(project_id, dataset_id, table_id, feature_data):
    """Streams a new record into the BigQuery log table."""
    bq_client = bigquery.Client(project=project_id)
    table_ref = bq_client.dataset(dataset_id).table(table_id)

    rows_to_insert = [feature_data] # The API expects a list of dictionaries

    errors = bq_client.insert_rows_json(table_ref, rows_to_insert)
    if not errors:
        print(f"Successfully saved feature {feature_data['feature_id']} to BigQuery.")
    else:
        print(f"❌ Error inserting rows to BigQuery: {errors}")

In [ ]:
def format_gemini_html_for_chat(html_content):
    """
    Converts complex HTML from Gemini into the limited HTML supported by Google Chat cards.

    - Converts <h2>, <h3>, <strong> to <b>
    - Converts <ul>/<li> into bulleted lists with <br>
    - Converts <hr> to '---'
    - Preserves <a> tags
    """
    if not html_content:
        return ""

    soup = BeautifulSoup(html_content, 'lxml')

    # 1. Replace headers (h2, h3, etc.) and <strong> with <b>
    for tag in soup.find_all(['h2', 'h3', 'h4', 'strong']):
        tag.name = 'b'
        # Add a line break after headers for spacing
        if tag.name in ['h2', 'h3', 'h4']:
            tag.insert_after(soup.new_tag('br'))

    # 2. Replace <hr>
    for tag in soup.find_all('hr'):
        tag.replace_with(BeautifulSoup('<br>---<br>', 'lxml'))

    # 3. Process <ul>
    for ul in soup.find_all('ul'):
        # Add a line break before the list
        ul.insert_before(soup.new_tag('br'))

        list_items_html = ""
        for li in ul.find_all('li'):
            # Get the inner content of li (which might include <a> tags)
            inner_html = li.decode_contents().strip()
            list_items_html += f"• {inner_html}<br>"

        # Replace the <ul> tag with the new HTML string, parsed by BS
        ul.replace_with(BeautifulSoup(list_items_html, 'lxml'))

    # 4. Handle <p> tags by just adding a line break after them and removing the tag
    for p in soup.find_all('p'):
        p.insert_after(soup.new_tag('br'))
        p.unwrap() # Remove the <p> tag itself, leaving content

    # 5. Get the final string.
    # We decode the contents of the body (if it exists) to avoid <html><body> tags
    if soup.body:
        formatted_html = soup.body.decode_contents()
    else:
        formatted_html = soup.decode_contents()

    # 6. Clean up potential excessive line breaks
    formatted_html = re.sub(r'(<br\s*/?>\s*){2,}', '<br>', formatted_html)

    return formatted_html.strip()

In [ ]:
def send_to_google_chat(feature_description, gemini_suggestion, published_date, link_url):
    """
    Sends a formatted card message to a Google Chat space using a webhook.
    """
    if not CHAT_WEBHOOK_URL:
        print("❌ Google Chat webhook URL was not retrieved from Secret Manager. Cannot send message.")
        return

    # --- 1. Clean the inputs for Google Chat Card formatting ---

    # Strip all HTML tags from the feature description to get plain text
    soup = BeautifulSoup(feature_description, 'lxml')
    cleaned_description = soup.get_text(separator=" ", strip=True)

    # Optional: Truncate the description if it's too long
    if len(cleaned_description) > 400:
        cleaned_description = cleaned_description[:400] + "..."

    # Convert Gemini's rich HTML into Chat-compatible HTML
    formatted_suggestion = format_gemini_html_for_chat(gemini_suggestion)

    # --- 2. Build the card with multiple widgets ---

    # Each item in this list is a separate widget in the card section
    widgets = [
        {
            "textParagraph": {
                "text": f"<b>New BigQuery Feature Published:</b><br>{cleaned_description}"
            }
        },
        {
            "textParagraph": {
                "text": f"{formatted_suggestion}"
            }
        },
        {
            "buttonList": {
                "buttons": [
                    {
                        "text": "View Release Note",
                        "onClick": {
                            "openLink": {
                                "url": link_url
                            }
                        }
                    }
                ]
            }
        }
    ]

    # --- 3. Build the full message body ---
    message_body = {
        'cardsV2': [{
            'cardId': 'release-note-suggestion-card',
            'card': {
                'header': {
                    'title': 'New BQ Feature & Onboarding Suggestions!',
                    'subtitle': f"Published on: {published_date}",
                    'imageUrl': 'https://cloud.google.com/images/social-icon-google-cloud-1200-630.png',
                    'imageType': 'CIRCLE'
                },
                'sections': [{
                    'widgets': widgets  # <-- Use the list of widgets here
                }]
            }
        }]
    }

    headers = {'Content-Type': 'application/json; charset=UTF-8'}
    try:
        response = requests.post(CHAT_WEBHOOK_URL, data=json.dumps(message_body), headers=headers)
        response.raise_for_status()
        print("✅ Card message sent to Google Chat successfully.")
    except requests.exceptions.RequestException as e:
        print(f"❌ Error sending message to Google Chat: {e}")

## 🚀 5. Run the Main Workflow

This final cell executes the entire process: fetch notes, generates suggestions, and send notifications.

In [ ]:
def main():
    """
    Orchestrates the entire process: fetch, generate, log, and send.
    """
    print(f"🚀 Starting BigQuery launch check for the last {LOOKBACK_DAYS} day(s)...")

    # Load the IDs of features we've already processed from BigQuery
    processed_ids = get_processed_ids_from_bq(GCP_PROJECT_ID, BIGQUERY_DATASET_ID, BIGQUERY_TABLE_ID)

    all_recent_features = get_recent_release_notes(days=LOOKBACK_DAYS)

    # Filter out features that have already been processed
    new_features_to_process = []
    for feature in all_recent_features:
        # Create a unique, repeatable ID for each feature based on its content
        feature_hash = hashlib.md5(feature['summary'].encode()).hexdigest()
        feature['id'] = f"{feature['published']}-{feature_hash}"

        if feature['id'] not in processed_ids:
            new_features_to_process.append(feature)

    if not new_features_to_process:
        print(f"✅ No new, unprocessed features found. All done.")
        return

    print(f"Found {len(new_features_to_process)} new feature(s) to process.")

    for i, launch in enumerate(new_features_to_process):
        print(f"\n[{i+1}/{len(new_features_to_process)}] Processing feature: {launch['id']}")
        use_case_ideas = generate_use_cases(launch)

        if use_case_ideas:
            # First, send the notification to Google Chat
            send_to_google_chat(
                feature_description=launch['summary'],
                gemini_suggestion=use_case_ideas,
                published_date=launch['published'],
                link_url=launch['link']
            )

            # Then, prepare the data and log the result to BigQuery
            record = {
                "feature_id": launch['id'],
                "published_date": launch['published'],
                "release_title": launch['title'],
                "feature_description": launch['summary'],
                "gemini_suggestion": use_case_ideas,
                "processing_timestamp": datetime.now(timezone.utc).isoformat()
            }
            save_result_to_bq(GCP_PROJECT_ID, BIGQUERY_DATASET_ID, BIGQUERY_TABLE_ID, record)

    print("\n🎉 Workflow complete.")

In [ ]:
#Run main
main()